## Name: Sadia Saad
## Email: sadiasaad2322@gmail.com
i have used a file dataset-mini.txt for checking the program repearedly.

In [61]:

import numpy as np
import pandas as pd
import copy
import nltk
from nltk.probability import ConditionalFreqDist

# File Reading
This cell reads the text file that will be used as our Corpus

In [62]:
#with open('dataset-mini.txt','r') as f:
with open('data.txt','r') as f:
    lines = f.readlines()
data = ""
# Appending all the lines into a string
for i in lines:
    data+=i
    
data=data.split('\n')



This cell below reads the misspellings.txt file it has a list of words and their misspellings

In [63]:
with open('misspellings.txt','r') as f:
    
    lines = f.readlines()
misspelled = ""
for i in lines:
    misspelled+=i
misspelled=misspelled.split('\n')
#misspelled
# Create a dictionary 
# It stores the correct word as key
# And the list of wrong words as its value.
misspellings = {}
firstRow = 1
for row in misspelled:
    
      if firstRow:
            firstRow = 0
      
      else:  
            # Slice the string read according to commas and tabs
            correct_word = row.split(',')[0]
            wrong_words = row.split(',')[-1]
            wrong_words = wrong_words[1:]
            wrong_words_list = wrong_words.split('\t')
            misspellings[correct_word] = wrong_words_list


# Unigram Model

In [64]:
# this list has all the words appearing in the vocabulary
# we read the string
# take a line
# split it
# and append the words in the uni-grams list.

unigrams = list()
for i in range(len(data)):
  words = data[i].split()
  for j in range(len(words)):
    w1 = words[j]
    unigrams.append(w1)

The cell below counts the **number of occurences of a word** and stores this value as a vlaue againt the word in a dictionary.
And the unique Words list has all the distinctive words that are actually keys of unigram_word_dictionary.

In [65]:
unigram_word_dictionary = {}
for w in unigrams:
  if unigram_word_dictionary.get(w):
    unigram_word_dictionary[w] += 1
  else:
    unigram_word_dictionary[w] = 1
global UniqueWords
UniqueWords = unigram_word_dictionary.keys()

## Unigram Model for Characters
Calculate count of characters existing in the vocabulary. Store it in a dictionary with **alphabets** as the key and **occurence** as the value

In [66]:
# Make Unigrams for the Characters
count_unigram_prob = {} # Empty dictionary 
for word in unigrams:
  for alphabet in word:
    if alphabet in count_unigram_prob: # If alphabet already exists in the dict
      count_unigram_prob[alphabet] += 1  #Add 1 to the occurance
    else: # If alphabet read for the first time
      count_unigram_prob[alphabet] = 1


## Bigram Model for Two Consecutive Words
Calculate count of characters that appear consecutively in the vocabulary. Store it in the dictionary, where the key is a tuple having two alphabets and the value is the number of times the two characters appeared consecutively in the vocabulary.

In [67]:
# Make Bigrams for the Characters
count_bigram = {} # Empty Dictionary
for correctWord in unigrams:
  correctWord = '#' + correctWord
  for x in range(len(correctWord) - 1): 
    w1 = correctWord[x]
    w2 = correctWord[x+1]
    if tuple((w1,w2)) in count_bigram: # If tuple already exists in the dict
      count_bigram[tuple((w1,w2))] += 1
    else: # If the tuple is read for the first time
      count_bigram[tuple((w1,w2))] = 1 

## Probability of Occurance of a word
It calculates the probability of a word using unigram word dictionary to return the probability of a word occuring in the corpus.
Its actually **Prior Probability P(w)**

In [68]:
# Language Model
# Train the Unigram model -> P(w)

def UnigramProbabilities(word): 
    
    if unigram_word_dictionary.get(word):
        return unigram_word_dictionary[word]/float(len(unigrams)) # Divide the Occurance with total # of words
    else:
        return -1


# Error Model
This model will be used to estimate the probability of typing x when w was
intended.
This error is modeled using character level insert, delete, transpose and substitute tables.

Function EditOperation takes two parameters. w represents the correct word and x represents the typed word. Returns the operation and x, y coordinates to be added in the confusion matrix 

**Logic Used in the Code:** Compare len of two strings.if **diff ==1** then operation is **insert or delete**. If diff of **length == 0** then operation is a **transpose or substitute**.
--- **For Transpose** Sort both words Compare them, if equal and the index of the misplaced word is **varied by 1** then it is transpose, else substitution.

In [69]:
def EditOperations(w,x):
  # If both words are different
  if w != x:
    w_list = list(w)
    x_list = list(x)
    # INSERT
    if len(w) <= len(x) - 1:
      Insert_found = 0
      for j in range(len(x_list) - 1):
        if x_list[j] != w_list[j] and j == 0:
          break
        if x_list[j] != w_list[j]:
          Insert_found = 1
          # The last letter of w is x and the current letter is y
          x = w_list[j-1]
          y = x_list[j]
          break
      if Insert_found:    
        return "insert", x, y
      else:
        # Insert operation at the first index
        if x_list[0] != w_list[0]:
          return "insert", '#', x_list[0]
        # Insert operation at the last index
        else:
          return "insert", w_list[-1], x_list[-1]

    # DELETE
    elif len(w) >= len(x) + 1:
      Delete_found = 0
      for i in range(len(w_list) - 1):
        if x_list[i] != w_list[i] and i==0:
          break
        if x_list[i] != w_list[i]:
          Delete_found = 1
          x = w_list[i-1]
          y = w_list[i]
          break
      if Delete_found:
        
        return "delete", x, y
      else:
        # Delete Operation at the first index
        if x_list[0] != w_list[0]:
          return "delete", '#', w_list[0]
        else:
          return 'delete', w_list[-2], w_list[-1]

    elif len(w) == len(x): # Either substitute or transpose
      copyW = copy.deepcopy(w)
      copyX = copy.deepcopy(x)
      # SUBSTITUTE
      if sorted(copyW) != sorted(copyX):  
        for i in range(len(w_list)):
            if w_list[i] != x_list[i]:
              return "substitute", x_list[i], w_list[i]
      # TRANSPOSE
      else:
        istrans = 0
        for a in range(len(x)-1):
          if x[a] == w[a+1] and x[a+1] == w[a]: # Check if indexes swapped are consecutive
            istrans = 1
            break
        if istrans:
          for i in range(len(w_list)-1):
            if w_list[i] != x_list[i]:
              return "transpose", w_list[i], w_list[i+1] 
  else:
    return "same", -1, -1

## Confusion Matrix
Making the Confusion Matrices for Insert, Delete, Transpose and Substitute Tables. Using the EditOperation function to poplulate the DataFrames

In [71]:

# X labels for the DataFrames
x_values = ['#','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
#Y labels for the DataFrames
y_values = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
# Create dataframes for the confusion matrices
# Insert DataFrame
Insert_DF = pd.DataFrame(columns = y_values, index = x_values)
Insert_DF.fillna(0, inplace = True) 

# Delete DataFrame
Delete_DF = pd.DataFrame(columns = y_values, index = x_values)
Delete_DF.fillna(0, inplace = True) 

# Substitue DataFrame
Substitute_DF = pd.DataFrame(columns = y_values, index = x_values)
Substitute_DF.fillna(0, inplace = True) 

#Transpose DataFrame
Transpose_DF = pd.DataFrame(columns = y_values, index = x_values)
Transpose_DF.fillna(0, inplace = True) 


for correctWord in misspellings:
  # Populate the confusion matrices using the edit operations
  for incorrectWord in misspellings[correctWord]:
    editOperation, x, y = EditOperations(correctWord,incorrectWord)
    if editOperation == 'insert':
        Insert_DF[y][x] += 1
    elif editOperation == 'delete':   
        Delete_DF[y][x] += 1
    elif editOperation == 'substitute':
        Substitute_DF[y][x] += 1
    elif editOperation == 'transpose':
        Transpose_DF[y][x] += 1

        
        
#..................................................

# Now printing all the dataframes

#...................................................

print("...Insert DataFrame...")
print(Insert_DF)
print("...Delete DataFrame...")
print(Delete_DF)
print("...Substitute DataFrame...")
print(Substitute_DF)
print("...Transpose DataFrame...")
print(Transpose_DF)


...Insert DataFrame...
     a    b    c    d    e    f    g    h    i    j  ...    q    r    s    t  \
#  594  650  705  682  710  707  683  655  707  671  ...  744  713  691  718   
a  548  693  644  652  662  641  656  608  577  629  ...  653  507  606  620   
b   57   81   95   97   60   86   66   81   91   83  ...   95   85   97   96   
c   55   64   69   66   58   67   68   31   58   63  ...   62   64   70   76   
d   99  109  127  136   97  148  114  111   98  130  ...  126  120  131  125   
e  273  298  248  230  235  272  270  267  267  277  ...  254  219  294  236   
f   23   39   58   38   32   44   38   37   32   41  ...   43   35   36   40   
g   56   88   88   80   74   74   74   58   80   91  ...   80   57   69   85   
h  112  206  210  237  185  228  234  217  192  228  ...  249  226  222  219   
i  229  254  259  252  250  246  248  261  244  288  ...  243  248  220  240   
j   38   53   53   42   50   50   49   47   47   68  ...   48   47   47   40   
k   88  123  119 

In [72]:
def getCountBigram(i, j):
  # Return the value against the tuple i, j in the count bigram
  return (count_bigram[i,j])

In [73]:
#Returns the number of character counts in the unigram calculated above 
# Count of individual alphabets in correct misspellings dictionary
def getCountOfWi(character):
    #NUMBER OF HASHES IN THE CORPUS IS EQUAL TO TOTAL NUMBER OF WORDS
  if character == '#':
    return len(unigrams)
  
  return count_unigram_prob[character] 

## Probability of a Word
ProbabilityOfWord() takes the correct word and the typed word. Computes P(x|w) by using probability according to theformulas of Insert, Delete, Substitute, and Transpose

In [74]:
def ProbabilityOfWord(w,x):
  operation, i, j = EditOperations(w,x)
  if operation == 'insert':
        
        probability = Insert_DF[j][i]/getCountOfWi(i)
    
  elif operation == 'delete':
    probability = Delete_DF[j][i]/getCountBigram(i,j)
  elif operation == 'substitute':
    probability = Substitute_DF[j][i]/getCountOfWi(j)
  elif operation == 'transpose':
    probability = Transpose_DF[j][i]/getCountBigram(i,j)
  elif operation == 'same':
    probability = 1
  return probability



In [75]:
# Function checks if two words have a transpose operation between them
def isTranspose(w,x):
  if sorted(w) == sorted(x):
    for a in range(len(x)-1):
      if x[a] == w[a+1] and x[a+1] == w[a]: # Checks if the alphabets were consecutive
        return True
  return False

## Levenshtein's Edit distance formula - to measure the difference between two sequences

In [76]:
# Calculate the edit distance between the given and candidate word
def EditDistance(w,x):
  len_w = len(w)
  len_x = len(x)
  edits = [[0 for e in range(len_x + 1)] for u in range(len_w + 1)]
  for i in range(len(w) + 1):
    for j in range(len(x) + 1):
      if i == 0:
        edits[i][j] = j
      elif j == 0:
        edits[i][j] = i
      elif w[i-1] == x[j-1]:
        edits[i][j] = edits[i-1][j-1]
      else:
        edits[i][j] = 1 + min(edits[i-1][j], edits[i][j-1], edits[i-1][j-1])
  return edits[len_w][len_x]


# Candidate Words Generation
Function CandidateWords, returns the candidate words with 1 edit distance with typed word.

In [77]:
def CandidateWords(x):
  candidateWords = []
  zeroEditCandidate = []
  # Check if the word exists in Vocab
  if x in UniqueWords:
    print("This word exists in the corpus.")
    #Return only that word.
    zeroEditCandidate.append(x)
    return zeroEditCandidate
  
  for correctWord in UniqueWords:
    #first calculate edit distance
    editDist = EditDistance(correctWord, x)
    #TRANSPOSE = 2 SUBSTITUTIONS
    isTrans = isTranspose(correctWord, x)
    if isTrans:
        editDist -= 1
    if editDist == 1:    
        candidateWords.append(correctWord)

  availableCandidates = []
  # in case the entire data is not read
  for w in candidateWords:
    try:
        
      p = UnigramProbabilities(w)
      if p > 0:
        availableCandidates.append(w)
    except:
      # garbage assignment
      abc=1 

  return availableCandidates



# Selection Model Using ArgMax

ProbableWord takes typed word and returns the highest probability word. The P(x|w) is computed from ProbabilityOfWord function and theP(w) is computed from UnigramProbabilities.Product is stored in options with  candidate word as key prob as value.

In [78]:

#will suggest the word with the maximum probability

def ProbableWord(x):    
  candidates = CandidateWords(x)
  options = {} # Dictionary
  for w in candidates:
    prob_xw = ProbabilityOfWord(w,x)  # P(x|w)
    prob_w = UnigramProbabilities(w)  # P(w)
    result = prob_xw * prob_w            #PRODUCT OF P(x|w) and P(w)
    options[w] = result                  
    print("Candidate: ", w, " Probability: ", result)
  if options:
    max_prob_word = max(options, key=options.get)
    return max_prob_word
  else:
    print("No Candidates Words are Found")
    return "None"


# Testing the Model on Different Words

In [79]:
x = 'rahk'
suggestion = ProbableWord(x)
print("Suggested Word: ", suggestion)

Candidate:  rahi  Probability:  5.714363459552975e-09
Candidate:  raha  Probability:  3.4873490902134255e-08
Candidate:  raho  Probability:  1.92977232130321e-09
Candidate:  rakh  Probability:  4.2728016699037896e-10
Candidate:  rahe  Probability:  3.8712884730063484e-09
Candidate:  rahy  Probability:  2.607132517817285e-10
Candidate:  rah  Probability:  1.5037833030451365e-10
Candidate:  rak  Probability:  8.015991623582741e-12
Candidate:  rashk  Probability:  4.8027012315556255e-11
Candidate:  rahu  Probability:  2.8704346909959607e-11
Candidate:  rank  Probability:  3.328832006705691e-11
Candidate:  rack  Probability:  1.795824306765435e-11
Candidate:  raak  Probability:  1.4278045606281783e-10
Suggested Word:  raha


In [80]:
x = 'khen'
suggestion = ProbableWord(x)
print("Suggested Word is: ",suggestion)

Candidate:  khan  Probability:  3.238659154419653e-10
Candidate:  khel  Probability:  1.6765211328720068e-10
Candidate:  hen  Probability:  2.7213032217948696e-10
Candidate:  kher  Probability:  8.501583850857543e-10
Candidate:  bhen  Probability:  3.802076497056538e-11
Candidate:  kahen  Probability:  4.192184365321231e-12
Candidate:  khena  Probability:  1.3619545816581951e-12
Candidate:  then  Probability:  1.5123070740888624e-10
Candidate:  kren  Probability:  9.250138408845782e-11
Candidate:  when  Probability:  7.488843875369143e-11
Candidate:  ehen  Probability:  3.7758291308943024e-11
Candidate:  khun  Probability:  1.0195337573951256e-11
Candidate:  ahen  Probability:  1.5579067311777295e-11
Candidate:  rhen  Probability:  2.5132697864542174e-11
Candidate:  khn  Probability:  3.3780497298110064e-12
Candidate:  khet  Probability:  5.7665518053291964e-12
Candidate:  khe  Probability:  2.1938990616075486e-11
Candidate:  khon  Probability:  8.708830773444552e-12
Candidate:  khin  

In [95]:
x = 'shukriay'   # This is a correct word so it should return the same word
suggestion = ProbableWord(x)
print("Suggested Word is: ",suggestion)

Candidate:  shukria  Probability:  2.7862602463549664e-11
Candidate:  shukriya  Probability:  2.114156842941423e-09
Suggested Word is:  shukriya


In [94]:
x = 'mukhtali'      # This is a correct word so it should return the same word
suggestion = ProbableWord(x)
print("Suggested Word is: ",suggestion)

Candidate:  mukhtalif  Probability:  5.857331928530717e-09
Candidate:  mukhtal  Probability:  7.06155658620652e-11
Candidate:  mukhtari  Probability:  1.4218742217196345e-11
Suggested Word is:  mukhtalif


In [83]:
#No candidate word will be found for this randomly created word
x = 'abcdefghi'
suggestion = ProbableWord(x)
print("Suggested Word is: ",suggestion)

No Candidates Words are Found
Suggested Word is:  None


In [84]:
x = 'pakistain'
suggestion = ProbableWord(x)
print("Suggested Word is: ",suggestion)

Candidate:  pakistan  Probability:  2.0166754158679666e-10
Candidate:  pakistani  Probability:  1.4435020961288475e-10
Suggested Word is:  pakistan


In [88]:
# Word does not return any candidate words
# Returns 'None' i.e. no word found
x = 'pichl'
suggestion = ProbableWord(x)
print("Suggested Word is: ",suggestion)

Candidate:  pichli  Probability:  2.2071845317137127e-10
Candidate:  pichla  Probability:  2.0521141972672746e-08
Candidate:  piche  Probability:  1.1965655696496032e-10
Candidate:  pichle  Probability:  3.456409175080245e-11
Candidate:  pichly  Probability:  1.7716282596503896e-11
Candidate:  picha  Probability:  6.597268647891886e-12
Candidate:  pichy  Probability:  1.8288928765313027e-12
Candidate:  pich  Probability:  3.0889373655479026e-11
Suggested Word is:  pichla
